In [1]:
import pandas as pd
from pathlib import Path

In [4]:
basedir = Path("/Users/jbirgmei/prog/sspsygene")
print(basedir)

/Users/jbirgmei/prog/sspsygene


In [9]:
mouse_perturb_dir = basedir / "data" / "pheno" / "data" / "mouse-perturb-4tf"
mouse_perturb_deg_file = mouse_perturb_dir / "deg.txt"
mouse_perturb_clusterprop_file = mouse_perturb_dir / "clusterprop.txt"


In [64]:
conversion_dict: dict[str, bool] = {
    "convert_string": True,
    "convert_integer": False,
    "convert_boolean": False,
    "convert_floating": False,
}
mouse_perturb_deg = pd.read_csv(mouse_perturb_deg_file, sep="\t").convert_dtypes(**conversion_dict)
mouse_perturb_clusterprop = pd.read_csv(mouse_perturb_clusterprop_file, sep="\t").convert_dtypes(**conversion_dict)



In [65]:
mouse_perturb_deg.head()

,Gene,logFC,logCPM,LR,PValue,padj,cell type,perturbation
0,Selenoo,9.651227,5.957813,19.984261,7.810000e-06,0.034711,Inhib_Lhx6+Sst-,Foxg1_3
1,Fbxl16,3.262595,7.123919,24.824542,6.280000e-07,0.005583,Inhib_Lhx6+Sst-,Foxg1_3
2,Meis2,-1.150044,7.671438,18.072991,2.130000e-05,0.027210,Excit_L5NP_CTX,Tbr1_1
3,Gm10762,8.258858,5.390421,17.808925,2.440000e-05,0.028133,Excit_L5NP_CTX,Tbr1_1
4,Uckl1,7.372006,5.647647,16.532829,4.780000e-05,0.034794,Excit_L5NP_CTX,Tbr1_1


In [66]:
mouse_perturb_clusterprop.head()

,subcluster,guide,PropMean.treatNonTarget2,PropMean.treatGuide,PropRatio,limma_coef,Tstatistic,P.Value,FDR
0,cluster_5,Foxg1_1,0.106732,0.010755,0.100762,-2.385959,-8.394585,6.410000e-07,0.000010
1,cluster_7,Foxg1_1,0.011199,0.273223,24.396342,3.587679,7.867436,4.150000e-05,0.000245
2,cluster_21,Foxg1_1,0.033080,0.006097,0.184311,-1.491792,-5.627320,5.630000e-05,0.000245
3,cluster_2,Foxg1_1,0.120282,0.003468,0.028834,-3.387877,-7.395180,6.530000e-05,0.000245
4,cluster_0,Foxg1_1,0.159987,0.047844,0.299048,-1.237609,-5.023114,1.717350e-04,0.000515


In [67]:
# print data types of columns:
print(mouse_perturb_deg.dtypes)
print(mouse_perturb_clusterprop.dtypes)


Gene            string[python]
logFC                  float64
logCPM                 float64
LR                     float64
PValue                 float64
padj                   float64
cell type       string[python]
perturbation    string[python]
dtype: object
subcluster                  string[python]
guide                       string[python]
PropMean.treatNonTarget2           float64
PropMean.treatGuide                float64
PropRatio                          float64
limma_coef                         float64
Tstatistic                         float64
P.Value                            float64
FDR                                float64
dtype: object


In [68]:
def sql_friendly_columns(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = df.columns.str.lower().str.replace(r'[^a-z0-9_]', '_', regex=True).str.replace(r'_+', '_', regex=True).str.strip('_')
    return df

mouse_perturb_deg = sql_friendly_columns(mouse_perturb_deg)
mouse_perturb_clusterprop = sql_friendly_columns(mouse_perturb_clusterprop)

In [69]:
def split_column(df: pd.DataFrame, source_col: str, new_col1: str, new_col2: str, sep: str = "_") -> pd.DataFrame:
    """
    Split `source_col` into two new columns (`new_col1`, `new_col2`) by `sep`,
    keeping the original column intact.
    """
    parts = df[source_col].astype("string").str.split(sep, n=1, expand=True)
    df[new_col1] = parts[0]
    df[new_col2] = parts[1]
    return df

# Example usage for mouse_perturb_deg:
split_column(
    mouse_perturb_deg,
    source_col="perturbation",
    new_col1="_perturbation_gene",
    new_col2="_perturbation_gene_idx",
    sep="_",
)

,gene,logfc,logcpm,lr,pvalue,padj,cell_type,perturbation,_perturbation_gene,_perturbation_gene_idx
0,Selenoo,9.651227,5.957813,19.984261,7.810000e-06,3.471145e-02,Inhib_Lhx6+Sst-,Foxg1_3,Foxg1,3
1,Fbxl16,3.262595,7.123919,24.824542,6.280000e-07,5.582916e-03,Inhib_Lhx6+Sst-,Foxg1_3,Foxg1,3
2,Meis2,-1.150044,7.671438,18.072991,2.130000e-05,2.720991e-02,Excit_L5NP_CTX,Tbr1_1,Tbr1,1
3,Gm10762,8.258858,5.390421,17.808925,2.440000e-05,2.813342e-02,Excit_L5NP_CTX,Tbr1_1,Tbr1,1
4,Uckl1,7.372006,5.647647,16.532829,4.780000e-05,3.479447e-02,Excit_L5NP_CTX,Tbr1_1,Tbr1,1
...,...,...,...,...,...,...,...,...,...,...
1622,Slc24a2,-2.450634,6.349151,43.731906,3.770000e-11,4.370000e-07,Excit_L5IT,Tcf4_2,Tcf4,2
1623,Nefl,-0.800898,7.212681,21.891280,2.890000e-06,1.198906e-02,Excit_L5IT,Tcf4_2,Tcf4,2
1624,Dcc,-0.804971,7.211753,21.752603,3.100000e-06,1.198906e-02,Excit_L5IT,Tcf4_2,Tcf4,2
1625,Ankrd50,-4.397718,6.327695,22.942305,1.670000e-06,1.650507e-02,CR,Tcf4_2,Tcf4,2


In [70]:
mouse_perturb_deg.head()

,gene,logfc,logcpm,lr,pvalue,padj,cell_type,perturbation,_perturbation_gene,_perturbation_gene_idx
0,Selenoo,9.651227,5.957813,19.984261,7.810000e-06,0.034711,Inhib_Lhx6+Sst-,Foxg1_3,Foxg1,3
1,Fbxl16,3.262595,7.123919,24.824542,6.280000e-07,0.005583,Inhib_Lhx6+Sst-,Foxg1_3,Foxg1,3
2,Meis2,-1.150044,7.671438,18.072991,2.130000e-05,0.027210,Excit_L5NP_CTX,Tbr1_1,Tbr1,1
3,Gm10762,8.258858,5.390421,17.808925,2.440000e-05,0.028133,Excit_L5NP_CTX,Tbr1_1,Tbr1,1
4,Uckl1,7.372006,5.647647,16.532829,4.780000e-05,0.034794,Excit_L5NP_CTX,Tbr1_1,Tbr1,1


In [71]:
split_column(
    mouse_perturb_clusterprop,
    source_col="guide",
    new_col1="_guide_gene",
    new_col2="_guide_gene_idx",
    sep="_",
)

,subcluster,guide,propmean_treatnontarget2,propmean_treatguide,propratio,limma_coef,tstatistic,p_value,fdr,_guide_gene,_guide_gene_idx
0,cluster_5,Foxg1_1,0.106732,0.010755,0.100762,-2.385959,-8.394585,6.410000e-07,0.000010,Foxg1,1
1,cluster_7,Foxg1_1,0.011199,0.273223,24.396342,3.587679,7.867436,4.150000e-05,0.000245,Foxg1,1
2,cluster_21,Foxg1_1,0.033080,0.006097,0.184311,-1.491792,-5.627320,5.630000e-05,0.000245,Foxg1,1
3,cluster_2,Foxg1_1,0.120282,0.003468,0.028834,-3.387877,-7.395180,6.530000e-05,0.000245,Foxg1,1
4,cluster_0,Foxg1_1,0.159987,0.047844,0.299048,-1.237609,-5.023114,1.717350e-04,0.000515,Foxg1,1
...,...,...,...,...,...,...,...,...,...,...,...
220,cluster_5,Tbr1_3,0.106732,0.104499,0.979078,-0.050764,-0.159583,8.737454e-01,0.989892,Tbr1,3
221,cluster_3,Tbr1_3,0.075759,0.076670,1.012021,-0.037493,-0.117863,9.065702e-01,0.989892,Tbr1,3
222,cluster_8,Tbr1_3,0.041878,0.043185,1.031206,0.036316,0.114165,9.094879e-01,0.989892,Tbr1,3
223,cluster_11,Tbr1_3,0.053383,0.053348,0.999344,-0.013988,-0.043973,9.650722e-01,0.989892,Tbr1,3


In [72]:
mouse_perturb_clusterprop.head()

,subcluster,guide,propmean_treatnontarget2,propmean_treatguide,propratio,limma_coef,tstatistic,p_value,fdr,_guide_gene,_guide_gene_idx
0,cluster_5,Foxg1_1,0.106732,0.010755,0.100762,-2.385959,-8.394585,6.410000e-07,0.000010,Foxg1,1
1,cluster_7,Foxg1_1,0.011199,0.273223,24.396342,3.587679,7.867436,4.150000e-05,0.000245,Foxg1,1
2,cluster_21,Foxg1_1,0.033080,0.006097,0.184311,-1.491792,-5.627320,5.630000e-05,0.000245,Foxg1,1
3,cluster_2,Foxg1_1,0.120282,0.003468,0.028834,-3.387877,-7.395180,6.530000e-05,0.000245,Foxg1,1
4,cluster_0,Foxg1_1,0.159987,0.047844,0.299048,-1.237609,-5.023114,1.717350e-04,0.000515,Foxg1,1


In [73]:
from processing.new_sqlite3 import NewSqlite3
import logging
logger = logging.getLogger(__name__)

with NewSqlite3("mouse_perturb.db", logger) as new_sqlite3:
    mouse_perturb_deg.to_sql("mouse_perturb_deg", new_sqlite3.conn, if_exists="replace", index=False)
    mouse_perturb_clusterprop.to_sql("mouse_perturb_clusterprop", new_sqlite3.conn, if_exists="replace", index=False)
